Competition: https://www.kaggle.com/c/competitive-data-science-predict-future-sales/overview 

# **Table of Contents** <a class="anchor" id="0.1"></a>

- [1. Data preprocessing ](#1)
    - [1.1 Анализ значений в наборе данных](#1.1)
    - [1.2 Data formating](#1.2)
- [2. LSTM](#2)
    - [2.1 Callbacks](#2.1)
    - [2.2 Model](#2.2)
    - [2.3 Custom Grid Search with cross-validation](#2.3)
    - [2.4 Использование оптимальных параметров для обучения конечной модели](#2.4)
    - [2.5 Prediction](#2.5)

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from tensorflow.keras.regularizers import L1L2
from tensorflow.keras.callbacks import EarlyStopping
import csv

In [2]:
sales_train = pd.read_csv("sales_train.csv")
items = pd.read_csv("items.csv")
item_categories = pd.read_csv('item_categories.csv')
shops = pd.read_csv("shops.csv")

test = pd.read_csv('test.csv')
sample_submission = pd.read_csv('sample_submission.csv')

In [3]:
print('Train data')
display(sales_train.head(2))
display(sales_train.shape)
print('Test data')
display(test.head(2))
display(test.shape)

Train data


,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.0,1.0
1,03.01.2013,0,25,2552,899.0,1.0


(2935849, 6)

Test data


,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320


(214200, 3)

## 1. Data preprocessing <a class="anchor" id="1"></a>

### 1.1 Анализ значений в наборе данных <a class="anchor" id="1.1"></a>

In [4]:
sales_train.describe()

,date_block_num,shop_id,item_id,item_price,item_cnt_day
count,2.935849e+06,2.935849e+06,2.935849e+06,2.935849e+06,2.935849e+06
mean,1.456991e+01,3.300173e+01,1.019723e+04,8.908532e+02,1.242641e+00
std,9.422988e+00,1.622697e+01,6.324297e+03,1.729800e+03,2.618834e+00
min,0.000000e+00,0.000000e+00,0.000000e+00,-1.000000e+00,-2.200000e+01
25%,7.000000e+00,2.200000e+01,4.476000e+03,2.490000e+02,1.000000e+00
50%,1.400000e+01,3.100000e+01,9.343000e+03,3.990000e+02,1.000000e+00
75%,2.300000e+01,4.700000e+01,1.568400e+04,9.990000e+02,1.000000e+00
max,3.300000e+01,5.900000e+01,2.216900e+04,3.079800e+05,2.169000e+03


Заметим, что в наборе данных присутствуют отрицательные цены на товары и отрицательные объемы продаж.

#### Наблюдения с отрицательным "item_cnt_day"

In [5]:
sales_train[sales_train.item_cnt_day < 0].shape

(7356, 6)

In [6]:
sales_train[sales_train.item_cnt_day < 0].item_cnt_day.value_counts()

-1.0     7252
-2.0       78
-3.0       14
-5.0        4
-4.0        3
-6.0        2
-9.0        1
-16.0       1
-22.0       1
Name: item_cnt_day, dtype: int64

In [7]:
sales_train[sales_train.item_cnt_day == 1].item_cnt_day.value_counts()

1.0    2629372
Name: item_cnt_day, dtype: int64

Отрицательные продажи - ошибка со знаком или нет?  
Тест различных преобразований отрицательных значений показал, что среди:  
- замены знака на положительный;
- зануления  

лучший score получаем при занулении

In [8]:
sales_train.loc[sales_train[sales_train.item_cnt_day < 0].index, "item_cnt_day"] = 0

#### Наблюдения с отрицательным "item_price"

In [9]:
temp = sales_train[sales_train.item_price < 0]
temp

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
484683,15.05.2013,4,32,2973,-1.0,1.0


In [10]:
test[(test.shop_id == temp.shop_id.values[0]) & (test.item_id == temp.item_id.values[0])].shape

(0, 3)

В тестовом наборе нет наблюдения с shop_id = 32 & item_id = 2973. Удалим все наблюдения с такой парой индексов.

In [11]:
sales_train.drop([484683], inplace=True)

### 1.2 Data formating <a class="anchor" id="1.2"></a>

train data (= sales_train) должны преобразовать так, чтобы индексами были shop_id & item_id.  
Поскольку стоит задача прогноза количества проданных экземпляров каждого продукта в каждом магазине за 34-ый месяц, посчитаем в каждом месяце общее число проданных экземпляров каждого продукта в каждом магазине. Для этого воспользуемся сводными таблицами ```pd.dataframe.pivot_table()```

In [12]:
dataset = sales_train.pivot_table(index = ['shop_id','item_id'],values = ['item_cnt_day'],columns = ['date_block_num'],fill_value = 0,aggfunc='sum')
display(dataset.head())
dataset.shape

item_cnt_day                              ...                 \
date_block_num            0   1  2  3  4  5  6  7  8  9   ... 24 25 26 27 28   
shop_id item_id                                           ...                  
0       30                 0  31  0  0  0  0  0  0  0  0  ...  0  0  0  0  0   
        31                 0  11  0  0  0  0  0  0  0  0  ...  0  0  0  0  0   
        32                 6  10  0  0  0  0  0  0  0  0  ...  0  0  0  0  0   
        33                 3   3  0  0  0  0  0  0  0  0  ...  0  0  0  0  0   
        35                 1  14  0  0  0  0  0  0  0  0  ...  0  0  0  0  0   

                                
date_block_num  29 30 31 32 33  
shop_id item_id                 
0       30       0  0  0  0  0  
        31       0  0  0  0  0  
        32       0  0  0  0  0  
        33       0  0  0  0  0  
        35       0  0  0  0  0  

[5 rows x 34 columns]

(424124, 34)

In [13]:
for i in range(len(dataset.columns)):
    dataset[dataset.columns[i]] = dataset[dataset.columns[i]].apply(lambda x: 0 if x < 0 else x)

Для обучения NN отберём только те наблюдения, пара индексов которых - shop_id & item_id, есть в тестовом наборе данных.

In [14]:
dataset = pd.merge(test,dataset,on = ['item_id','shop_id'],how = 'left')
dataset.fillna(0,inplace = True)
dataset_for_test = dataset.drop(['ID'], axis = 1)
dataset = dataset.drop(['shop_id','item_id','ID'], axis = 1)

/home/user/anaconda3/lib/python3.8/site-packages/pandas/core/reshape/merge.py:618: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)


с data scaling результаты хуже. Возможно при scaling data стоило подобрать другой learning_rate 

In [15]:
def data_reshaping(x_train, y_train):
    x_train = x_train.astype("float32")# / dataset.values.max() 
    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1],  1))
    y_train = y_train.astype("float32")# / dataset.values.max()
    return(x_train, y_train)

def data_separation(x, target, with_validation=True):
    if(with_validation): 
        x_train, x_val, y_train, y_val = train_test_split(x, target, test_size=0.2, 
                                                          random_state=42, shuffle = True)
        right_number_batches_train = (x_train.shape[0] // batch_size) * batch_size
        right_number_batches_val = (x_val.shape[0] // batch_size) * batch_size
        x_train, x_val, y_train, y_val = x_train[:right_number_batches_train], x_val[:right_number_batches_val], y_train[:right_number_batches_train], y_val[:right_number_batches_val]
        return(x_train, x_val, y_train, y_val)
    else:
        x_train, y_train = x, target
        right_number_batches_train = (x_train.shape[0] // batch_size) * batch_size
        x_train, y_train = x_train[:right_number_batches_train], y_train[:right_number_batches_train]
        return(x_train, y_train)

def data_reshaping_test(x_test):
    x_test = x_test.astype("float32")# / dataset.values.max()
    x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1],  1))
    return(x_test)

## 2. LSTM <a class="anchor" id="2"></a>

### 2.1 Callbacks <a class="anchor" id="2.1"></a>

In [16]:
'''
early_stopping_callback have a bug. To get rid of it, create own class, which inheritates early_stopping_callback
taken from [ https://github.com/tensorflow/tensorflow/issues/35634#issuecomment-665517890 ]
 to get rid of bug when early_stopping_callback doesn't return best weights.
'''
class ReturnBestEarlyStopping(EarlyStopping):
    def __init__(self, **kwargs):
        super(ReturnBestEarlyStopping, self).__init__(**kwargs)

    def on_train_end(self, logs=None):
        if self.stopped_epoch > 0:
            if self.verbose > 0:
                print(f'\nEpoch {self.stopped_epoch + 1}: early stopping')
        elif self.restore_best_weights:
            if self.verbose > 0:
                print('Restoring model weights from the end of the best epoch.')
            self.model.set_weights(self.best_weights)

class LossAndErrorPrintingCallback(tf.keras.callbacks.Callback):
    
    def on_test_end(self, logs=None):
        print('Validation | Average losses: {:.3e}. MSE {:.1e}.'.format(logs['loss'],logs['mse']))
        
    def on_epoch_end(self, epoch, logs=None):#Вызывается в конце эпохи во время ОБУЧЕНИЯ.
        print('Training | Average losses: {:.3e}. MSE {:.1e}.'.format(logs['loss'], logs['mse']))

save_callback = tf.keras.callbacks.ModelCheckpoint(
    "checkpoint/checkpoint-{epoch:02d}", save_weights_only=False, monitor="loss", save_best_only=False,
)

lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(
    monitor="loss", factor=0.7, patience=7, mode="max", verbose=1, min_lr=0.0001, cooldown=1
)
tensorboard_callback = keras.callbacks.TensorBoard(
    log_dir="tb_callback_dir", histogram_freq=1,
)

### 2.2 Model <a class="anchor" id="2.2"></a>

In [17]:
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

def LSTM(batch_size, regularization_set, dropout, learning_rate):
    model = keras.Sequential()
    model.add(keras.Input(batch_input_shape=(batch_size, time_steps, 1)))
    
    model.add(
        layers.LSTM(64, return_sequences=False, activation="tanh",stateful=False, dropout=dropout, recurrent_regularizer=regularization_set)#stateful=False
    )
    model.add(layers.Dense(1))
    
    model.compile(
        loss='mse',
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
        metrics=["mse"],
    )
    return(model)

def model_fitting(x_train, x_val, y_train, y_val, regularization_set, 
                  learning_rate, batch_size, dropout, n_epoch=10, with_validation=True):

    model = LSTM(batch_size, regularization_set, dropout, learning_rate)

    if(with_validation):
        model.fit(x_train, y_train, batch_size=batch_size , epochs=n_epoch, verbose=0, shuffle=True,
                  validation_data=(x_val, y_val),
                  callbacks=[LossAndErrorPrintingCallback(),
                             ReturnBestEarlyStopping(monitor='mse', min_delta=0, patience=6, verbose=0, mode='auto',baseline=None, restore_best_weights=True)]) 
    else:
        model.fit(x_train, y_train, batch_size=batch_size , epochs=30, verbose=1, shuffle=True,
                 callbacks=[tf.keras.callbacks.ModelCheckpoint(monitor='loss', verbose=1, save_best_only=True, filepath="saved_weights/")]) 

    return(model)

### 2.3 Custom Grid Search with cross-validation <a class="anchor" id="2.3"></a>

In [18]:
def cross_validation_separation(k_fold):
    x_val = x[k_fold*right_number_batches_val : (k_fold+1)*right_number_batches_val]
    y_val = target[k_fold*right_number_batches_val : (k_fold+1)*right_number_batches_val]
    
    x_train = np.delete(x, range(k_fold*right_number_batches_val, (k_fold+1)*right_number_batches_val), axis = 0)
    x_train = x_train[:right_number_batches_train]
    y_train = np.delete(target, range(k_fold*right_number_batches_val, (k_fold+1)*right_number_batches_val),axis = 0)
    y_train = y_train[:right_number_batches_train]
    return(x_train, x_val, y_train, y_val)

def get_mse_score(model, batch_size, x, y):
    output = model.predict(x, batch_size=batch_size)
    mse_score = mean_squared_error(y, output)
    return(mse_score)

def cross_validation(k_folds, reg, lr, batch_size, dropout, n_epoch=10):
    validation_mse_list, train_mse_list = list(), list()
    for k_fold in range(k_folds):
        print(f"k_fold: {k_fold+1}/{k_folds}")
        x_train, x_val, y_train, y_val = cross_validation_separation(k_fold)
        model = model_fitting(x_train, x_val, y_train, y_val, regularization_set=reg, 
                              learning_rate=lr, batch_size=batch_size, dropout=dropout, n_epoch=n_epoch)
        
        validation_mse = get_mse_score(model, batch_size, x_val, y_val)
        validation_mse_list.append(validation_mse)
        train_mse = get_mse_score(model, batch_size, x_train, y_train)
        train_mse_list.append(train_mse)
    return(validation_mse_list, train_mse_list)

In [ ]:
regularization_weights = [L1L2(l1=0.0, l2=0.0), L1L2(l1=0.01, l2=0.0), L1L2(l1=0.005, l2=0.0)]
batch_size_list = [1000, 2000] 
learning_rates = [0.0005, 0.001]
dropout_list = [0.0, 0.2, 0.4]

time_steps = 33
grid_search_dict = {}
model_list = list()

k_folds = 5
target = dataset.values[:,-1]
x = dataset.values[:,:-1]
x, target = data_reshaping(x, target)

for batch_size in batch_size_list:
    print(f"\n\n\n ----------- batch_size: {batch_size} -----------")
    right_number_batches_train = int(( (dataset.shape[0]*(k_folds-1)/k_folds) // batch_size) * batch_size)
    right_number_batches_val = int( ( (dataset.shape[0]*(1)/k_folds) // batch_size) * batch_size)
    for lr in learning_rates:
        for reg in regularization_weights:  
            for dropout in dropout_list:
                reg_key = (f'batch_size {batch_size}, learning_rate {lr}, dropout {dropout} , l1 {np.round(reg.l1.item(),3)}, l2 {np.round(reg.l2.item(),3)}')
                
                validation_mse_list, train_mse_list = cross_validation(k_folds, reg, lr, batch_size, dropout)
                validation_mse_mean = np.mean(validation_mse_list)
                train_mse_mean = np.mean(train_mse_list)
                print(f"validation_mse_mean: {round(validation_mse_mean,3)}. train_mse_mean: {round(train_mse_mean,3)}")
                grid_search_dict[reg_key] = f"val_mse_mean:{validation_mse_mean} & train_mse_mean: {train_mse_mean} || validation_mse_list: {validation_mse_list} | train_mse_list: {train_mse_list}"#min(model.history.history["val_mse"])

In [ ]:
def save_dict_to_file(dic, name):
    f = open(f'{name}.txt','w+')
    f.write(str(dic))
    f.close()

def load_dict_from_file(name):
    f = open(f'grid_search_result/{name}.txt','r')
    data=f.read()
    f.close()
    return eval(data)

def save_dict_in_csv(name):
    dict_1 = load_dict_from_file(name)
    with open(f'{name}.csv', 'w+') as f:
        sorted_dict = dict(sorted(dict_1.items(), key = lambda x: x[1])[:])
        for key in sorted_dict.keys():
            f.write("%s,%s\n"%(key,sorted_dict[key]))
        
def update_dict(name_1, name_2):
    dict_1 = load_dict_from_file(name_1)
    dict_2 = load_dict_from_file(name_2)
    dict_1.update(dict_2)
    save_dict_to_file(dict_1, f'{name_1}_{name_2}')

def top_5_optimal_params(file_names):
    print('Top 5 sets model parameters with MSE score on validation data')
    for f_name in file_names:
        gs_dict = load_dict_from_file(f_name)
        display(dict(sorted(gs_dict.items(), key = lambda x: x[1])[:5]))

save_dict_to_file(grid_search_dict, 'GS_Cross_validation_not_full_WithOUT_zero_deletion')

### 2.4 Использование оптимальных параметров для обучения конечной модели  <a class="anchor" id="2.4"></a>

Cross-validation of model with optimal params

In [19]:
reg_optimal = L1L2(l1=0.0, l2=0.0)
batch_size_optimal = 2000
batch_size = batch_size_optimal
lr_optimal = 0.0005
dropout_optimal = 0.2

n_epoch = 7
k_folds = 5
time_steps = 33
target = dataset.values[:,-1]
x = dataset.values[:,:-1]
x, target = data_reshaping(x, target)

right_number_batches_train = int(( (dataset.shape[0]*(k_folds-1)/k_folds) // batch_size) * batch_size)
right_number_batches_val = int( ( (dataset.shape[0]*(1)/k_folds) // batch_size) * batch_size)


validation_mse_list, train_mse_list = cross_validation(k_folds, reg_optimal, lr_optimal, batch_size, dropout_optimal, n_epoch)
print(f"validation_mse_list: {validation_mse_list}, \ntrain_mse_list: {train_mse_list}")
print(f"validation_mse_mean: {np.mean(validation_mse_list)}, \ntrain_mse_mean: {np.mean(train_mse_list)}")

k_fold: 1/5
Validation | Average losses: 1.293e+02. MSE 1.3e+02.
Training | Average losses: 6.385e+00. MSE 6.4e+00.
Validation | Average losses: 1.289e+02. MSE 1.3e+02.
Training | Average losses: 6.165e+00. MSE 6.2e+00.
Validation | Average losses: 1.286e+02. MSE 1.3e+02.
Training | Average losses: 6.055e+00. MSE 6.1e+00.
Validation | Average losses: 1.284e+02. MSE 1.3e+02.
Training | Average losses: 5.979e+00. MSE 6.0e+00.
Validation | Average losses: 1.282e+02. MSE 1.3e+02.
Training | Average losses: 5.918e+00. MSE 5.9e+00.
Validation | Average losses: 1.281e+02. MSE 1.3e+02.
Training | Average losses: 5.874e+00. MSE 5.9e+00.
Validation | Average losses: 1.280e+02. MSE 1.3e+02.
Training | Average losses: 5.817e+00. MSE 5.8e+00.
k_fold: 2/5
Validation | Average losses: 1.435e+01. MSE 1.4e+01.
Training | Average losses: 3.490e+01. MSE 3.5e+01.
Validation | Average losses: 1.405e+01. MSE 1.4e+01.
Training | Average losses: 3.471e+01. MSE 3.5e+01.
Validation | Average losses: 1.383e+01. 

Заметим, что ошибка что на training data, что и на validation data почти монотонно уменьшается.  
Создадим модель, которая будет предсказывать значения для тестового набора данных.  
P.S.: в данном случае создаётся модель без валидационной выборки, поскольку на kaggle есть тестовый набор данных, на котором модель будет апробирована.

In [20]:
reg_optimal = L1L2(l1=0.0, l2=0.0)
batch_size_optimal = 2000
batch_size = batch_size_optimal
lr_optimal = 0.0005
dropout_optimal = 0.2
time_steps = 33


target = dataset.values[:,-1]
x = dataset.values[:,:-1]
x, target = data_reshaping(x, target)

x_train, y_train = data_separation(x, target, with_validation=False)

model = LSTM(batch_size, reg_optimal, dropout_optimal, lr_optimal)

model.fit(x_train, y_train, batch_size=batch_size , epochs=10, verbose=0, shuffle=True,
          callbacks=[LossAndErrorPrintingCallback(), save_callback, tensorboard_callback])#,lr_scheduler]) 

train_mse = get_mse_score(model, batch_size, x_train, y_train)
print(f"train MSE:{train_mse}")
#display(model.history.history)

Instructions for updating:
use `tf.profiler.experimental.stop` instead.
Training | Average losses: 3.052e+01. MSE 3.1e+01.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: checkpoint/checkpoint-01/assets
Training | Average losses: 3.024e+01. MSE 3.0e+01.
INFO:tensorflow:Assets written to: checkpoint/checkpoint-02/assets
Training | Average losses: 3.006e+01. MSE 3.0e+01.
INFO:tensorflow:Assets written to: checkpoint/checkpoint-03/assets
Training | Average losses: 2.995e+01. MSE 3.0e+01.
INFO:tensorflow:Assets written to: checkpoint/checkpoint-04/assets
Training | Average losses: 2.989e+01. MSE 3.0e+01.
INFO:tensorflow:Assets written to: checkpoint/checkpoint-05/assets
Training | Average losses: 2.984e+01. MSE 3.0e+01.
INFO:tensorflow:Assets written to: checkpoint/checkpo

### 2.5 Prediction <a class="anchor" id="2.5"></a>

In [21]:
n_batch = 700

reg_optimal = L1L2(l1=0.0, l2=0.0)
batch_size_optimal = 2000
batch_size = batch_size_optimal
lr_optimal = 0.0005
dropout_optimal = 0.2
time_steps = 33


target = dataset.values[:,-1]
x = dataset.values[:,:-1]
x, target = data_reshaping(x, target)

new_model = LSTM(n_batch, reg_optimal, dropout_optimal, lr_optimal)
new_model.load_weights(f'checkpoint/checkpoint-1.02548/variables/variables') 

x_train, y_train = data_separation(x, target, with_validation=False)
predicted_values = new_model.predict(x_train, batch_size=batch_size).flatten()
train_mse = mean_squared_error(y_train, predicted_values)
print(f"MSE on training: {train_mse}")

def get_model_for_test(model, n_batch = 1):#if wanna use model to predict value, for example,for one observation
    # re-define model
    new_model = LSTM(n_batch, reg_optimal, dropout_optimal, lr_optimal)
    # copy weights
    old_weights = model.get_weights()
    new_model.set_weights(old_weights)
    return(new_model)

MSE on training: 29.528867721557617


In [22]:
x_test = pd.merge(test,dataset_for_test,on = ['item_id','shop_id'],how = 'left')
print(x_test.shape)
x_test.fillna(0,inplace = True)
display(x_test.head())
test_id = x_test.pop('ID')
x_test.drop(['shop_id','item_id'], axis=1, inplace=True)
x_test.drop(x_test.columns[0], axis=1, inplace=True)
x_test.head()

(214200, 37)


,ID,shop_id,item_id,"(item_cnt_day, 0)","(item_cnt_day, 1)","(item_cnt_day, 2)","(item_cnt_day, 3)","(item_cnt_day, 4)","(item_cnt_day, 5)","(item_cnt_day, 6)",...,"(item_cnt_day, 24)","(item_cnt_day, 25)","(item_cnt_day, 26)","(item_cnt_day, 27)","(item_cnt_day, 28)","(item_cnt_day, 29)","(item_cnt_day, 30)","(item_cnt_day, 31)","(item_cnt_day, 32)","(item_cnt_day, 33)"
0,0,5,5037,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,0.0,0.0,0.0,1.0,1.0,1.0,3.0,1.0,0.0
1,1,5,5320,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,5,5233,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,3.0,2.0,0.0,1.0,3.0,1.0
3,3,5,5232,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,4,5,5268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


,"(item_cnt_day, 1)","(item_cnt_day, 2)","(item_cnt_day, 3)","(item_cnt_day, 4)","(item_cnt_day, 5)","(item_cnt_day, 6)","(item_cnt_day, 7)","(item_cnt_day, 8)","(item_cnt_day, 9)","(item_cnt_day, 10)",...,"(item_cnt_day, 24)","(item_cnt_day, 25)","(item_cnt_day, 26)","(item_cnt_day, 27)","(item_cnt_day, 28)","(item_cnt_day, 29)","(item_cnt_day, 30)","(item_cnt_day, 31)","(item_cnt_day, 32)","(item_cnt_day, 33)"
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,0.0,0.0,0.0,1.0,1.0,1.0,3.0,1.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,3.0,2.0,0.0,1.0,3.0,1.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [23]:
x_test = data_reshaping_test(x_test.values)

In [24]:
#new_model = get_model_for_test(model, n_batch)
predicted_values = new_model.predict(x_test, batch_size=n_batch).flatten()
submission = pd.DataFrame({'ID':test_id.values,'item_cnt_month':predicted_values})
submission.to_csv('LSTM_full.csv',index = False)

Kaggle result: Score: 1.02548 (50% в leaderboard [4818/9684])  
https://www.kaggle.com/konstantinlp/competitions